# Based on cb_galex_sedfit.py

### Import relevant packages
### note: need the following --> pip install pysynphot

In [4]:
import numpy as np
import os
import argparse
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from sedpy import observate
import pysynphot as S
from uncertainties import ufloat


/home/idies/miniconda3/lib/python3.7/site-packages/pysynphot/locations.py:47: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/home/idies/miniconda3/lib/python3.7/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


### Expand Pandas data tables to show all values

In [29]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Can be commented out or run as necessary

### List of templates, names of columns, colors

In [24]:
templates = ['Composite1', 'Composite2', 'Composite3', 'AGN1', 'AGN2', 'AGN3', 'AGN4', 'SFG1', 'SFG2', 'SFG3',
             'IR_COLOR1', 'IR_COLOR2', 'IR_COLOR3', 'IR_COLOR4', 'IR_COLOR5', 'IR_COLOR6', 'IR_COLOR7', 'IR_COLOR8']

# This is a combination of the Libraries for High Z Dusty Galaxies and AGN
# It uses all templates from the Color-Based Library and all but Composite 4
# from the Comprehensive Library

fits_cols = ['galaxy', 'template_name', 'filter', 'mags', 'wave_eff', 'model_phot', 'model_phot_mags','diff']

colors = {'Composite1': 'silver', 'Composite2': 'rosybrown', 'Composite3': 'darksalmon',
        'AGN1': 'cornflowerblue', 'AGN2': 'blue', 'AGN3': 'slateblue', 'AGN4': 'paleturquoise',
        'SFG1': 'blueviolet', 'SFG2': 'plum', 'SFG3': 'mediumorchid',
        'IR_COLOR1': 'olive', 'IR_COLOR2': 'olivedrab', 'IR_COLOR3': 'yellowgreen', 'IR_COLOR4': 'greenyellow',
        'IR_COLOR5': 'lawngreen', 'IR_COLOR6': 'lightgreen', 'IR_COLOR7': 'darkgreen', 'IR_COLOR8': 'aquamarine'}


### Open table

In [25]:
table = fits.open('hizea_photo_galex_wise_v1.0.fit')

### Set up variables

In [26]:
cols = ['fuv', 'nuv', 'u', 'g', 'r', 'i', 'z', 'w1', 'w2', 'w3', 'w4',
        'fuv_unc', 'nuv_unc', 'u_unc', 'g_unc', 'r_unc', 'i_unc', 'z_unc', 'w1_unc', 'w2_unc', 'w3_unc', 'w4_unc', 'Z']
filt_waves = table[1].data['PHOT_WAVE'][0].byteswap().newbyteorder()*(10**-4)
# PHOT_WAVE finds central wavelength of each photometric data point given in angstroms

gals_redshifts = np.array([[i] for i in table[1].data['Z']])
np.array([[i] for i in table[1].data['Z']])
gal_names = table[1].data['SHORT_NAME'].byteswap().newbyteorder()


### Function to convert between f_lambda and f_nu

In [27]:
def flam_to_fnu(flux):
    spec = S.ArraySpectrum(table[1].data['PHOT_WAVE'][0], flux, fluxunits='Flam')
    spec.convert('Fnu')
    return spec.flux

### Make a table with fluxes and errors

In [28]:
flam = table[1].data['FLUX_FLAM'].byteswap().newbyteorder()
fnu = np.array([flam_to_fnu(flammie) for flammie in flam])
flamu = table[1].data['FLUX_FLAM_ERR'].byteswap().newbyteorder()
fnuu = np.array([flam_to_fnu(flammieu) for flammieu in flamu])
flux_w_err = np.concatenate((fnu, fnuu, gals_redshifts), axis=1)
gals_flux = pd.DataFrame(data=flux_w_err,
                        index=gal_names,
                        columns=cols)


gals_flux


#print(gals_flux.loc[gal_names, :][:11].values)

Warning, 1 of 11 bins contained negative fluxes; they have been set to zero.


,fuv,nuv,u,g,r,i,z,w1,w2,w3,w4,fuv_unc,nuv_unc,u_unc,g_unc,r_unc,i_unc,z_unc,w1_unc,w2_unc,w3_unc,w4_unc,Z
J0106-1023,8.086701e-12,5.152274e-11,1.165052e-10,1.549535e-10,2.653417e-10,2.978942e-10,3.149088e-10,5.130973e-10,3.736707e-10,1.670159e-09,3.754655e-09,5.086919e-12,6.326321e-12,3.528212e-12,1.517740e-12,2.088619e-12,2.786244e-12,8.711742e-12,7.266125e-12,1.623130e-11,2.212796e-10,1.382886e-09,0.454389
J0315-0740,3.603791e-12,4.848827e-11,5.963875e-11,8.689124e-11,2.281569e-10,2.612095e-10,3.078958e-10,4.342246e-10,2.850591e-10,3.436287e-10,3.135281e-10,9.351906e-12,1.059417e-11,3.373788e-12,1.291198e-12,2.107316e-12,2.676355e-12,8.642657e-12,6.233206e-12,1.240395e-11,1.223921e-10,1.209939e-09,0.460214
J0811+4716,0.000000e+00,2.026833e-11,2.500427e-11,3.535234e-11,9.424071e-11,1.257363e-10,1.544866e-10,2.024498e-10,1.101401e-10,4.279289e-10,1.250847e-09,0.000000e+00,6.499420e-12,2.200399e-12,1.029321e-12,1.502833e-12,1.939381e-12,7.023733e-12,6.550682e-12,1.323534e-11,1.595699e-10,1.122595e-09,0.515353
J0824+5032,1.549452e-12,6.521493e-12,2.040648e-11,3.964637e-11,5.998919e-11,1.000751e-10,1.284337e-10,3.845255e-10,5.030435e-10,7.232788e-10,3.787612e-09,1.115738e-12,1.223557e-12,2.166236e-12,1.070440e-12,1.477962e-12,2.026068e-12,7.427505e-12,7.607378e-12,1.512347e-11,1.827216e-10,1.307662e-09,0.779669
J0826+4305,6.972604e-12,5.243078e-11,9.680237e-11,1.102961e-10,1.387254e-10,1.731988e-10,1.708487e-10,3.354324e-10,2.343445e-10,1.155559e-09,4.799140e-09,1.518230e-12,2.068977e-12,3.073060e-12,1.148240e-12,1.756678e-12,2.303144e-12,7.183739e-12,7.668162e-12,1.442104e-11,1.804158e-10,1.300996e-09,0.603152
J0827+2954,6.480714e-13,9.112145e-12,1.935729e-11,2.654737e-11,5.936839e-11,1.107644e-10,1.376322e-10,2.575184e-10,1.657390e-10,3.691256e-10,1.936365e-09,1.454147e-12,1.919298e-12,2.505706e-12,1.051069e-12,1.643910e-12,2.261499e-12,8.167993e-12,8.448336e-12,1.640380e-11,1.944216e-10,1.368627e-09,0.681294
J0901+0314,0.000000e+00,0.000000e+00,1.066798e-10,1.212293e-10,1.757576e-10,1.916592e-10,2.072619e-10,3.525933e-10,2.420118e-10,8.847248e-10,3.695654e-09,0.000000e+00,0.000000e+00,2.871354e-12,1.299211e-12,1.713299e-12,2.186054e-12,6.510664e-12,7.684184e-12,1.555721e-11,1.816796e-10,1.299599e-09,0.458913
J0905+5759,1.458397e-12,3.312964e-11,7.873443e-11,9.953515e-11,1.141881e-10,1.481593e-10,1.366396e-10,2.438968e-10,1.622877e-10,9.577173e-10,3.817113e-09,1.179505e-12,1.941089e-12,2.868704e-12,1.298911e-12,1.704200e-12,2.237923e-12,9.006737e-12,6.617107e-12,1.301876e-11,1.586770e-10,1.163403e-09,0.711434
J0908+1039,0.000000e+00,0.000000e+00,8.024965e-11,1.021788e-10,1.896615e-10,2.358197e-10,2.745405e-10,5.002771e-10,3.583490e-10,6.086654e-10,3.543959e-10,0.000000e+00,0.000000e+00,2.321737e-12,1.115239e-12,1.613770e-12,2.306701e-12,6.274241e-12,7.310877e-12,1.498249e-11,1.636016e-10,1.141167e-09,0.502308
J0933+5614,0.000000e+00,1.336307e-11,3.365835e-11,5.695091e-11,1.672738e-10,2.402104e-10,2.808981e-10,5.071850e-10,3.646439e-10,8.211404e-10,3.818363e-09,0.000000e+00,4.694159e-12,2.231865e-12,1.007439e-12,1.685597e-12,2.466545e-12,7.373011e-12,6.444611e-12,1.308309e-11,1.478326e-10,1.063133e-09,0.504228


In [22]:
gals_flux.loc[:,'w4']# prints out all data points for a specific column

J0106-1023    3.754655e-09
J0315-0740    3.135281e-10
J0811+4716    1.250847e-09
J0824+5032    3.787612e-09
J0826+4305    4.799140e-09
J0827+2954    1.936365e-09
J0901+0314    3.695654e-09
J0905+5759    3.817113e-09
J0908+1039    3.543959e-10
J0933+5614    3.818363e-09
J0939+4251    8.149058e-10
J0944+0930    4.727238e-09
J1036-0102    1.152716e-09
J1039+4537             NaN
J1052+0607    1.623733e-09
J1052+4104             NaN
J1104+5946    2.462848e-10
J1107+0417    5.685502e-09
J1125-0145    2.786575e-09
J1133+0956    3.909341e-10
J1142+6037    1.578301e-09
J1205+1818    9.259683e-09
J1219+0336    8.470236e-09
J1229+3545    1.888418e-10
J1232+0723             NaN
J1235+6140    1.965683e-11
J1239+0731    6.112137e-10
J1244+4140    1.266028e-09
J1248+0601    6.697179e-10
J1341-0321    7.066341e-09
J1359+5137    1.461708e-09
J1450+4621    1.731395e-09
J1500+1739    4.942304e-09
J1506+6131    4.483756e-10
J1506+5402    1.056356e-08
J1516+1650    1.395329e-09
J1558+3957    3.913407e-09
J

### Make a table with magnitudes and errors

In [11]:
mags = table[1].data['AB_MAG'].byteswap().newbyteorder()
magsu = table[1].data['AB_MAG_ERR'].byteswap().newbyteorder()
mags_w_err = np.concatenate((mags, magsu, gals_redshifts), axis=1)
gals_mag = pd.DataFrame(data=mags_w_err,
                        index=gal_names,
                        columns=cols)

### Functions

In [12]:
# ----- Read the template into a data frame. Name columns something convenient.
def read_template(name):
    temps_path = 'kirkpatrick/'
    temp = pd.read_csv(temps_path+name+'.txt',
                       names=['rest_wavelength','luminosity','DLnu'],
                       skiprows=[0, 1, 2, 3],
                       sep=r'\s{1,}',
                       engine='python')
    return temp

# ----- Function to match a target wavelength's position.
def mask_wave(temp_wavel, target_wave):
    return np.abs(temp_wavel-target_wave) == np.amin(np.abs(temp_wavel-target_wave))

# ----- Designed to take in flux and return mags
def mag(flux):
    mag = -2.5*np.log10(flux)
    return mag

### Read the Galaxy Files

In [50]:
j1039 = fits.open('J1039+4537DR13sdsswise.fits')
j1039[1].data


#print(j1039[1].data['w4_mag_err'])

print(j1039[1].data['w4_mag_err'].len)


AttributeError: 'HDUList' object has no attribute 'size'

In [124]:
data = galaxy[1].data

first_two_rows= data[:2]

first_two_rows


#data = fits.getdata('J1039+4537sdsswise.fits', 1)


#t = Table(data)






FITS_rec([( 1, 200.36294732, 10.4372342 , 0.00136899, 0.00150094, '1237671956440809490', 555.97675, 809.23816, 0, 1, '1999p106',  870.08594, 0.0057923 , 15.151094, 0.01639595,  2.7245367, 0.94509774, 3.6740515e-05, 0.03196772, 135., 30.329178,  798.7183, 0.00041075, 15.244016 , 0.06707184, 0.59859526, 0.91694  , 4.0438084e-05, 0.03229877,  91., 29.881332, -1672.7188, 1.1450721e-07,        nan,      nan, 0.6054282 , 0.73825485, 0.00309962, 5.184789, 65., 15.478139, -3462.426, 1.7136559e-10,       nan,      nan, 0.4061607, 0.8124564 , 0.09592014, 332.11633, 177., 17.233843, 6166, 2, 30),
          ( 1, 200.41401386, 10.45759111, 0.00069035, 0.00079287, '1237671956440809493', 490.2644 , 835.9689 , 0, 1, '1999p106', 4531.3877 , 0.00595511, 13.359422, 0.0031049 , 51.794094 , 0.975867  , 1.3225888e-04, 0.59931564, 406., 31.48535 , 4298.459 , 0.00040533, 13.4167185, 0.01254609, 5.7701645 , 0.9631514, 6.3603868e-05, 0.27339855, 223., 30.86924 ,  3807.541 , 1.0933113e-07, 13.5483885, 0.862398, 

### Code from ad_table_example

In [35]:
# determining fluxes and uncertainties for W4
w4_nanomaggies = j1039[1].data['w4_nanomaggies'][0]
w4_nanomaggies_ivar = j1039[1].data['w4_nanomaggies_ivar'][0]
w4_unc = 1. / np.sqrt(w4_nanomaggies_ivar)
w4_snr = w4_nanomaggies / w4_unc
print(w4_nanomaggies, w4_unc, w4_snr)

-3462.426 76390.29405234726 -0.045325470576379256


In [36]:
# what if we took the W4 uncertainty as an upper limit on the flux?
w4_upper_1sigma = w4_unc
w4_mag_upper_1sigma = -2.5*np.log10(w4_upper_1sigma*1e-9)
print(w4_mag_upper_1sigma)

10.292404545553067


In [37]:
# what if we treated the negative value as the flux?
w4_mag_perhaps = -2.5*np.log10(np.abs(w4_nanomaggies)*1e-9)
print(w4_mag_perhaps)

13.651548741286494


In [38]:
# difference between upper limit and "value"
w4_err = w4_mag_perhaps - w4_mag_upper_1sigma
print(w4_err)
# so maybe we quote this as 11.73 +/- 1.53 in terms of Vega magnitude?

3.3591441957334265


In [40]:
# check values related to chi-squared
w4_prochi2 = j1039[1].data['w4_prochi2'][0] # Profile-weighted chi-squared
w4_pronpix = j1039[1].data['w4_pronpix'][0] # number-of-pixels values for chi-squared calculation
print(w4_prochi2, w4_pronpix)

0.4061607 0.8124564


In [42]:
# check info about nearby sources
w4_proflux = j1039[1].data['w4_proflux'][0] #  profile-weighted, the amount of flux contributed by other nearby sources.
w4_profracflux = j1039[1].data['w4_profracflux'][0] #  equal to w4 proflux divided by w4 nanomaggies -- fraction of flux from other sources
print(w4_proflux, w4_profracflux, w4_proflux/w4_profracflux)

332.11633 0.09592014 3462.4255


In [125]:
galaxy.close()

### Fit templates, return column to feed into a new table

In [39]:
'''def sed_fitting(gal_name, template_name):
    z = gals_mag.loc[gal_name, 'Z']
    template = tempsdf[tempsdf.template_name == template_name]
    
    # ----- Organizing wavelength and luminosity
    z_temp_wavel = template.rest_wavelength * (1 + z)
    gal_fluxes = gals_flux.loc[gal_name, :][:10].values
    W2_wavelength = filt_waves[8]
    # Figure out where the template lines up with W2
    mask = mask_wave(z_temp_wavel, W2_wavelength)
    
    # Scale template to match value at W2
    factor = gal_fluxes[8] / float(template.luminosity[mask].values[0])
    luminosity = template.luminosity * factor  # Scale

    # ----- Readying wavelength and flux for sedpy
    wave_aa = np.array(z_temp_wavel[0:-1]) * 1e4  # changes angstroms to microns
    flux = np.array(luminosity[0:-1])
    fnu = flux * 3631. * 1e-23 # converts from angstroms to microns?
    flambda = fnu * 2.998e18 / (wave_aa) ** 2 # see flux.doc in the Drive for math explanation

    # ----- Using sedpy to get wise band photometry based on templates
    filternames = ['wise_w{}'.format(n) for n in ['1', '2', '3', '4']]
    wise_filters = observate.load_filters(filternames)
    model_mags = observate.getSED(wave_aa, flambda, filterlist=wise_filters)
    wave_eff = [f.wave_effective for f in wise_filters]
    model_phot = 10. ** (model_mags / (-2.5))# converts from magnitudes to flux??
    
   # diff = model_phot[1]/float(template.luminosity[mask].values[0])
    diff = model_phot[1]/gal_fluxes[8]

    rows = pd.DataFrame([[gal_name, template_name, filternames[i],
                          model_mags[i], wave_eff[i], model_phot[i],
                          mag(model_phot[i])] for i in range(len(filternames))],
                        columns=fits_cols)
    return rows'''

In [56]:
def sed_fitting(gal_name, template_name):
    z = gals_mag.loc[gal_name, 'Z']
    template = tempsdf[tempsdf.template_name == template_name]

    # ----- Organizing wavelength and luminosity
    z_temp_wavel = template.rest_wavelength * (1 + z)
    gal_fluxes = gals_flux.loc[gal_name, :][:10].values
    W2_wavelength = filt_waves[8]
    # Figure out where the template lines up with W2
    mask = mask_wave(z_temp_wavel, W2_wavelength)
    
    # Scale template to match value at W2
    factor = gal_fluxes[8] / float(template.luminosity[mask].values[0])
    luminosity = template.luminosity * factor  # Scale

    # ----- Readying wavelength and flux for sedpy
    wave_aa = np.array(z_temp_wavel[0:-1]) * 1e4  # changes angstroms to microns
    flux = np.array(luminosity[0:-1])
    fnu = flux * 3631. * 1e-23 # converts from angstroms to microns?
    flambda = fnu * 2.998e18 / (wave_aa) ** 2 # see flux.doc in the Drive for math explanation

    # ----- Using sedpy to get wise band photometry based on templates
    filternames = ['wise_w{}'.format(n) for n in ['1', '2', '3', '4']]
    wise_filters = observate.load_filters(filternames)
    model_mags = observate.getSED(wave_aa, flambda, filterlist=wise_filters)
    wave_eff = [f.wave_effective for f in wise_filters]
    model_phot = 10. ** (model_mags / (-2.5))# converts from magnitudes to flux??
    
   # diff = model_phot[1]/float(template.luminosity[mask].values[0])
    diff = model_phot[1]/gal_fluxes[8]

    rows = pd.DataFrame([[gal_name, template_name, filternames[i],
                          model_mags[i], wave_eff[i], model_phot[i],
                          mag(model_phot[i]), diff] for i in range(len(filternames))],
                        columns=fits_cols)
    return rows

### Plots templates for comparison and calculates lowest chi

In [50]:
'''def template_comparison(gal_name, template_name):
    z = gals_mag.loc[gal_name, 'Z']
    template = tempsdf[tempsdf.template_name == template_name].reset_index(drop=True)[['rest_wavelength', 'luminosity']]
    z_temp_wavel = template.rest_wavelength * (1 + z)
    gal_fluxes = gals_flux.loc[gal_name, :][:11].values
    W3_wavelength = filt_waves[9]
    # Figure out where the template lines up with W1
    mask = mask_wave(z_temp_wavel, W3_wavelength)
    # Scale template to match value at W1
    factor = gal_fluxes[9]/float(template.luminosity[mask].values[0])
    luminosity = template.luminosity*factor # Scale
    model_phot = sed_fits[(sed_fits.galaxy == gal_name) & (sed_fits.template_name == template_name)].model_phot.array

    gal_unc = gals_flux.iloc[:,11:-1].loc[gal_name].values
    chi = np.sum(np.array([((gal_fluxes[i + 8] - model_phot[i + 1]) / gal_unc[i + 8]) ** 2 for i in range(3)])) / 3

    # Plot
    plot = True

    if plot:
        plot_color = colors[template_name]
        title = gal_name + '-' + template_name

        g = sns.lineplot(x=z_temp_wavel, y=luminosity, color=plot_color, label=template_name, alpha=0.6, ax=ax)
        h = sns.scatterplot(x=filt_waves, y=gal_fluxes, ax=ax, color='blue')
        # ax.scatter(x=np.array(wave_eff)/1e4, y=model_phot, color='red', s=11)
        ax.errorbar(filt_waves, gal_fluxes, yerr=gal_unc, color='blue', ls='none')
        ax.set_ylim([1e-14, 1e-7])
        ax.set_xlim([0.1, 1000.])
        ax.loglog()
        ax.legend()
        ax.set_title(title)
        plt.ioff()
        # plt.savefig(title+'.png')
        # plt.clf()
        # plt.close()

    return (chi)'''

In [57]:
def template_comparison(gal_name, template_name):
    z = gals_mag.loc[gal_name, 'Z']
    template = tempsdf[tempsdf.template_name == template_name].reset_index(drop=True)[['rest_wavelength', 'luminosity']]
    z_temp_wavel = template.rest_wavelength * (1 + z)
    gal_fluxes = gals_flux.loc[gal_name, :][:10].values
    W2_wavelength = filt_waves[8]
    # Figure out where the template lines up with W2
    mask = mask_wave(z_temp_wavel, W2_wavelength)
    # Scale template to match value at W2
    factor = gal_fluxes[8]/float(template.luminosity[mask].values[0])
    luminosity = template.luminosity*factor
    # Scale and match luminosity of rest wavelength with template
    model_phot = sed_fits[(sed_fits.galaxy == gal_name) & (sed_fits.template_name == template_name)].model_phot.array
    
    #

    gal_unc = gals_flux.iloc[:,11:-1].loc[gal_name].values
    chi = np.sum(np.array([((gal_fluxes[i + 8] - model_phot[i + 1]) / gal_unc[i + 8]) ** 2 for i in range(3)])) / 3

    # Plot
    plot = True

    if plot:
        plot_color = colors[template_name]
        title = gal_name + '-' + template_name

        g = sns.lineplot(x=z_temp_wavel, y=luminosity, color=plot_color, label=template_name, alpha=0.6, ax=ax)
        h = sns.scatterplot(x=filt_waves, y=gal_fluxes, ax=ax, color='blue')
        # ax.scatter(x=np.array(wave_eff)/1e4, y=model_phot, color='red', s=11)
        ax.errorbar(filt_waves, gal_fluxes, yerr=gal_unc, color='blue', ls='none')
        ax.set_ylim([1e-14, 1e-7])
        ax.set_xlim([0.1, 1000.])
        ax.loglog()
        ax.legend()
        ax.set_title(title)
        plt.ioff()
        # plt.savefig(title+'.png')
        # plt.clf()
        # plt.close()

    return (chi)
# Whichever template has the smallest chi value is assigned to the galaxy

### Make general plots showing all galaxies (different colors and different filters)

In [58]:
def general_color_plots():
    # Different filters
    fig = plt.figure(figsize=(17, 10))
    ax = fig.add_subplot(1,1,1)
    w2 = sed_fits[sed_fits['filter'] == 'wise_w2'].model_phot_mags.array
    w3 = sed_fits[sed_fits['filter'] == 'wise_w3'].model_phot_mags.array
    w4 = sed_fits[sed_fits['filter'] == 'wise_w4'].model_phot_mags.array
    sns.scatterplot(x=w3 - w4,
                    y=w2 - w3,
                    data=sed_fits.iloc[::4, :].reset_index(), hue='template_name', ax=ax)
    ax.set_ylabel('W3-W4')
    ax.set_xlabel('W2-W3')
    ax.set_title('Color vs color - filters')
    plt.savefig('se_galex_color_byfilt.png')
    plt.close('all')
    # Different galaxies
    fig = plt.figure(figsize=(17, 10))
    ax = fig.add_subplot(1,1,1)
    w2 = sed_fits[sed_fits['filter'] == 'wise_w2'].model_phot.array
    w3 = sed_fits[sed_fits['filter'] == 'wise_w3'].model_phot.array
    w4 = sed_fits[sed_fits['filter'] == 'wise_w4'].model_phot.array
    sns.scatterplot(x=mag(w3) - mag(w4),
                    y=mag(w2) - mag(w3),
                    data=sed_fits.iloc[::4, :].reset_index(), hue='galaxy', palette='Paired')
    ax.set_ylabel('W3-W4')
    ax.set_xlabel('W2-W3')
    ax.set_title('Color vs color - galaxies')
    plt.savefig('se_galex_color_bygal.png')
    plt.close('all')


### Making color vs color plots for individual galaxies

In [59]:
def color_plots():
    print(' ---------> Making color plots showing different filters...')
    with PdfPages('se_galex_sed_fitting_colorplots.pdf') as pdf:
        for galaxy in gal_names:
            print("---Making color plot: ", galaxy)
            fig = plt.figure(figsize=(17, 10))
            ax = fig.add_subplot(1,1,1)
            sedf_g = sed_fits.copy()[sed_fits.galaxy == galaxy].reset_index(drop=True)

            w2 = sedf_g[sedf_g['filter'] == 'wise_w2'].model_phot_mags.array
            w3 = sedf_g[sedf_g['filter'] == 'wise_w3'].model_phot_mags.array
            w4 = sedf_g[sedf_g['filter'] == 'wise_w4'].model_phot_mags.array
            w2_gal = gals_mag['w2'][galaxy]
            w3_gal = gals_mag['w3'][galaxy]
            w4_gal = gals_mag['w4'][galaxy]
            
            w2_unc = ufloat(gals_mag['w2'][galaxy], gals_mag['w2_unc'][galaxy])
            w3_unc = ufloat(gals_mag['w3'][galaxy], gals_mag['w3_unc'][galaxy])
            w4_unc = ufloat(gals_mag['w4'][galaxy], gals_mag['w4_unc'][galaxy])
            
            sns.scatterplot(x=w3 - w4, y=w2 - w3,
                            data=sedf_g[sedf_g['filter'] == 'wise_w2'], hue='template_name', ax=ax)

            ax.plot(w3_gal - w4_gal, w2_gal - w3_gal, marker='*', markersize=14, label=galaxy)
            ax.errorbar((w3_unc-w4_unc).n, (w2_unc-w3_unc).n,xerr= (w3_unc-w4_unc).s, yerr=(w2_unc-w3_unc).s, color='blue', ls='none')
            # plots both vertical and horizontal errorbars
            ax.set_title(galaxy)
            ax.set_ylabel('W2-W3')
            ax.set_xlabel('W3-W4')
            pdf.savefig(bbox_inches="tight")
            plt.close('all')
        print('-----Finished color plots pdf')

### Making a table with the templates

In [60]:
print('Reading templates into data frame...')
tempsdf = pd.DataFrame([],columns=['rest_wavelength','luminosity','DLnu'])
for temp_name in templates:
    newdf = read_template(temp_name)
    newdf['template_name'] = [temp_name for i in range(newdf.shape[0])]
    tempsdf = tempsdf.append(newdf)
print('Templates read.')

Reading templates into data frame...
Templates read.


### Generating fits for galaxies

In [61]:
sed_fits = pd.DataFrame([], columns=fits_cols)
print(' ---------> Fitting templates to data...')
for gal in gal_names:
    print("---Fitting ", gal)
    for tem in templates:
        sed_fits = sed_fits.append(sed_fitting(gal, tem))
print('---Finished fitting templates to data.\n')
sed_fits.reset_index(inplace=True, drop=True)

 ---------> Fitting templates to data...
---Fitting  J0106-1023
---Fitting  J0315-0740
---Fitting  J0811+4716
---Fitting  J0824+5032
---Fitting  J0826+4305
---Fitting  J0827+2954
---Fitting  J0901+0314
---Fitting  J0905+5759
---Fitting  J0908+1039
---Fitting  J0933+5614
---Fitting  J0939+4251
---Fitting  J0944+0930
---Fitting  J1036-0102
---Fitting  J1039+4537
---Fitting  J1052+0607
---Fitting  J1052+4104
---Fitting  J1104+5946
---Fitting  J1107+0417
---Fitting  J1125-0145
---Fitting  J1133+0956
---Fitting  J1142+6037
---Fitting  J1205+1818
---Fitting  J1219+0336
---Fitting  J1229+3545
---Fitting  J1232+0723
---Fitting  J1235+6140
---Fitting  J1239+0731
---Fitting  J1244+4140
---Fitting  J1248+0601
---Fitting  J1341-0321
---Fitting  J1359+5137
---Fitting  J1450+4621
---Fitting  J1500+1739
---Fitting  J1506+6131
---Fitting  J1506+5402
---Fitting  J1516+1650
---Fitting  J1558+3957
---Fitting  J1604+3939
---Fitting  J1611+2650
---Fitting  J1613+2834
---Fitting  J1622+3145
---Fitting  J163

In [62]:
sed_fits = pd.DataFrame([], columns=fits_cols)
print(' ---------> Fitting templates to data...')
for gal in gal_names:
    print("---Fitting ", gal)
    for tem in templates:
        sed_fits = sed_fits.append(sed_fitting(gal, tem))
print('---Finished fitting templates to data.\n')
sed_fits.reset_index(inplace=True, drop=True)

 ---------> Fitting templates to data...
---Fitting  J0106-1023
---Fitting  J0315-0740
---Fitting  J0811+4716
---Fitting  J0824+5032
---Fitting  J0826+4305
---Fitting  J0827+2954
---Fitting  J0901+0314
---Fitting  J0905+5759
---Fitting  J0908+1039
---Fitting  J0933+5614
---Fitting  J0939+4251
---Fitting  J0944+0930
---Fitting  J1036-0102
---Fitting  J1039+4537
---Fitting  J1052+0607
---Fitting  J1052+4104
---Fitting  J1104+5946
---Fitting  J1107+0417
---Fitting  J1125-0145
---Fitting  J1133+0956
---Fitting  J1142+6037
---Fitting  J1205+1818
---Fitting  J1219+0336
---Fitting  J1229+3545
---Fitting  J1232+0723
---Fitting  J1235+6140
---Fitting  J1239+0731
---Fitting  J1244+4140
---Fitting  J1248+0601
---Fitting  J1341-0321
---Fitting  J1359+5137
---Fitting  J1450+4621
---Fitting  J1500+1739
---Fitting  J1506+6131
---Fitting  J1506+5402
---Fitting  J1516+1650
---Fitting  J1558+3957
---Fitting  J1604+3939
---Fitting  J1611+2650
---Fitting  J1613+2834
---Fitting  J1622+3145
---Fitting  J163

### Make color vs color plots

In [63]:
color_plots()
print('-----Generated se_galex_sed_fitting_colorplots.pdf with color vs color plots.')

 ---------> Making color plots showing different filters...
---Making color plot:  J0106-1023
---Making color plot:  J0315-0740
---Making color plot:  J0811+4716
---Making color plot:  J0824+5032
---Making color plot:  J0826+4305
---Making color plot:  J0827+2954
---Making color plot:  J0901+0314
---Making color plot:  J0905+5759
---Making color plot:  J0908+1039
---Making color plot:  J0933+5614
---Making color plot:  J0939+4251
---Making color plot:  J0944+0930
---Making color plot:  J1036-0102
---Making color plot:  J1039+4537
---Making color plot:  J1052+0607
---Making color plot:  J1052+4104
---Making color plot:  J1104+5946
---Making color plot:  J1107+0417
---Making color plot:  J1125-0145
---Making color plot:  J1133+0956
---Making color plot:  J1142+6037
---Making color plot:  J1205+1818
---Making color plot:  J1219+0336
---Making color plot:  J1229+3545
---Making color plot:  J1232+0723
---Making color plot:  J1235+6140
---Making color plot:  J1239+0731
---Making color plot: 

In [58]:
general_color_plots()
print('----Generated se_galex_color_byfilt.png and se_galex_color_bygal.png showing color vs color for all gals.')

----Generated galex_color_byfilt.png and galex_color_bygal.png showing color vs color for all gals.


### Produce a PDF file showing template fits

In [59]:
# this takes some time to run (about a minute per galaxy?)
with PdfPages('galexsed_fitting.pdf') as pdf:
    print('\n ---------> Plotting templates and calculating chi values...\n')
    for gal in gal_names:
        fig = plt.figure(figsize=(17,10))
        ax = fig.add_subplot(1,1,1)
        print("---Fitting ", gal)
        chis = []
        for tem in templates:
            new_chi = template_comparison(gal, tem)
            chis.append(new_chi)
        bestchi_pos = chis.index(min(chis))
        result_string = "Galaxy - " + gal + " - lowest chi template: " + str(templates[bestchi_pos])
        print(result_string)
        plt.text(10, 10**-6.5, result_string, ha='center')
        pdf.savefig(bbox_inches="tight")
        plt.close('all')
    print('Finished!')


 ---------> Plotting templates and calculating chi values...

---Fitting  J0106-1023


/home/idies/miniconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Galaxy - J0106-1023 - lowest chi template: AGN4
---Fitting  J0315-0740
Galaxy - J0315-0740 - lowest chi template: AGN3
---Fitting  J0811+4716
Galaxy - J0811+4716 - lowest chi template: AGN2
---Fitting  J0824+5032
Galaxy - J0824+5032 - lowest chi template: AGN1
---Fitting  J0826+4305
Galaxy - J0826+4305 - lowest chi template: AGN4
---Fitting  J0827+2954
Galaxy - J0827+2954 - lowest chi template: AGN1
---Fitting  J0901+0314
Galaxy - J0901+0314 - lowest chi template: AGN4
---Fitting  J0905+5759
Galaxy - J0905+5759 - lowest chi template: AGN4
---Fitting  J0908+1039
Galaxy - J0908+1039 - lowest chi template: AGN3
---Fitting  J0933+5614
Galaxy - J0933+5614 - lowest chi template: AGN3
---Fitting  J0939+4251
Galaxy - J0939+4251 - lowest chi template: AGN3
---Fitting  J0944+0930
Galaxy - J0944+0930 - lowest chi template: IR_COLOR2
---Fitting  J1036-0102
Galaxy - J1036-0102 - lowest chi template: IR_COLOR3
---Fitting  J1039+4537
Galaxy - J1039+4537 - lowest chi template: Composite1
---Fitting  J

/home/idies/miniconda3/lib/python3.7/site-packages/matplotlib/axes/_axes.py:3265: RuntimeWarning: invalid value encountered in double_scalars
  high = [thisx + thiserr for (thisx, thiserr)


Galaxy - J1052+4104 - lowest chi template: Composite1
---Fitting  J1104+5946
Galaxy - J1104+5946 - lowest chi template: AGN3
---Fitting  J1107+0417
Galaxy - J1107+0417 - lowest chi template: IR_COLOR1
---Fitting  J1125-0145
Galaxy - J1125-0145 - lowest chi template: IR_COLOR3
---Fitting  J1133+0956
Galaxy - J1133+0956 - lowest chi template: AGN3
---Fitting  J1142+6037
Galaxy - J1142+6037 - lowest chi template: AGN3
---Fitting  J1205+1818
Galaxy - J1205+1818 - lowest chi template: IR_COLOR2
---Fitting  J1219+0336
Galaxy - J1219+0336 - lowest chi template: Composite3
---Fitting  J1229+3545
Galaxy - J1232+0723 - lowest chi template: Composite1
---Fitting  J1235+6140
Galaxy - J1235+6140 - lowest chi template: AGN3
---Fitting  J1239+0731
Galaxy - J1239+0731 - lowest chi template: AGN3
---Fitting  J1244+4140
Galaxy - J1244+4140 - lowest chi template: IR_COLOR8
---Fitting  J1248+0601
Galaxy - J1248+0601 - lowest chi template: AGN3
---Fitting  J1341-0321
Galaxy - J1341-0321 - lowest chi templa